# Generic Rule Induction Notebook

Continuoulsy refined.

In [50]:
import sys
sys.path.append('../../')
import aix360i.algorithms.rule_induction.r2n.simple_rulenet as rn
import aix360i.algorithms.rule_induction.r2n.base as base
import aix360i.algorithms.rule_induction.r2n.utilities as util
import aix360i.algorithms.rule_induction.r2n.preprocessing_layers as ul
import aix360i.algorithms.rule_induction.r2n.r2n_algo as algo
from aix360i.algorithms.rule_induction.r2n.training import train as train_R2N

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [51]:
from config import config_dict
from config import config_dict_imbalanced

# document config order
CONFIG = config_dict_imbalanced["CONFIG-I63"]
#CONFIG = config_dict["CONFIG53"]




print('Proceed with configuration:', CONFIG["NAME"])

Proceed with configuration: german-credit-brcg


In [52]:
import pandas as pd
import numpy as np
import seaborn as sns
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import matthews_corrcoef,fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import wittgenstein as lw
import time
import warnings
import re

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREES':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper
if CONFIG['ALGO'] == 'CORELS':
    from corels import *



# from explainer import Explainer

# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


### Data

In [53]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG':
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)
    CONFIG['POS_CLASS'] = 1
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df[CONFIG['TARGET_LABEL']].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Status Checking          1000 non-null   object
 1   Duration in Month        1000 non-null   int64 
 2   Credit History           1000 non-null   object
 3   Purpose                  1000 non-null   object
 4   Credit Amount            1000 non-null   int64 
 5   Savings Account          1000 non-null   object
 6   Employement since        1000 non-null   object
 7   Installmentrate %        1000 non-null   int64 
 8   StatusSex                1000 non-null   object
 9   Otherdebtos              1000 non-null   object
 10  PresentResidence         1000 non-null   int64 
 11  Property                 1000 non-null   object
 12  Age in years             1000 non-null   int64 
 13  Otherinstallment Plans   1000 non-null   object
 14  Housing                  1000 non-null   

1    700
0    300
Name: Target, dtype: int64

In [54]:

# BRCG Miniloan sample
#if CONFIG['UNDERSAMP'] == True:
#    under_sampling_fraud = df[df[CONFIG['TARGET_LABEL']] == 1].sample(n=4000, random_state = 42)
#    under_size = under_sampling_fraud.size
#    under_sampling_legit = (df[df[CONFIG['TARGET_LABEL']] == 0])
#    df = pd.concat([under_sampling_legit, under_sampling_fraud])

#Ripper Miniloan sample
#if CONFIG['UNDERSAMP'] == True:
#    under_sampling_fraud = df[df[CONFIG['TARGET_LABEL']] == 1]
#    under_size = under_sampling_fraud.size
#    under_sampling_legit = (df[df[CONFIG['TARGET_LABEL']] == 0].sample(n=4000, random_state = 42))
#    df = pd.concat([under_sampling_legit, under_sampling_fraud])

In [55]:
CONFIG['POS_CLASS']

0

In [56]:
if CONFIG['TYPE'] == 'BINARY':
    print(df[CONFIG['TARGET_LABEL']].value_counts())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    df[CONFIG['TARGET_LABEL']].describe()
else:
    print('Unrecognized problem type')

1    700
0    300
Name: Target, dtype: int64


### Train, Test Split

In [57]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'],random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (700, 20) (700,)
Test: (300, 20) (300,)


In [58]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 541 to 102
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Status Checking          700 non-null    object
 1   Duration in Month        700 non-null    int64 
 2   Credit History           700 non-null    object
 3   Purpose                  700 non-null    object
 4   Credit Amount            700 non-null    int64 
 5   Savings Account          700 non-null    object
 6   Employement since        700 non-null    object
 7   Installmentrate %        700 non-null    int64 
 8   StatusSex                700 non-null    object
 9   Otherdebtos              700 non-null    object
 10  PresentResidence         700 non-null    int64 
 11  Property                 700 non-null    object
 12  Age in years             700 non-null    int64 
 13  Otherinstallment Plans   700 non-null    object
 14  Housing                  700 non-null   

### Reference Performance

In [59]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

### Binarization

In [60]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
#x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 541 to 102
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Status Checking          700 non-null    object
 1   Duration in Month        700 non-null    int64 
 2   Credit History           700 non-null    object
 3   Purpose                  700 non-null    object
 4   Credit Amount            700 non-null    int64 
 5   Savings Account          700 non-null    object
 6   Employement since        700 non-null    object
 7   Installmentrate %        700 non-null    int64 
 8   StatusSex                700 non-null    object
 9   Otherdebtos              700 non-null    object
 10  PresentResidence         700 non-null    int64 
 11  Property                 700 non-null    object
 12  Age in years             700 non-null    int64 
 13  Otherinstallment Plans   700 non-null    object
 14  Housing                  700 non-null   

,Status Checking,Duration in Month,Credit History,Purpose,Credit Amount,Savings Account,Employement since,Installmentrate %,StatusSex,Otherdebtos,PresentResidence,Property,Age in years,Otherinstallment Plans,Housing,Number existing Credits,Job,Number people liable,Telephone,Foreign Worker
541,A14,24,delay in paying off in the past,car (new),2032,< 100DM,>= 7 years,4,male : single,None,4,unknown/no property,60,None,for free,2,skilled employee /official,1,"yes, registered under the customers name",yes
440,A14,12,existing credits paind back duly till now,car (new),1884,< 100DM,>= 7 years,4,male : single,None,4,"if not others options : car or other, not in s...",39,None,own,1,management/self-employed/highly qualified empl...,1,"yes, registered under the customers name",yes
482,A11,30,existing credits paind back duly till now,furniture/equipment,3622,>= 1000 DM,>= 7 years,4,female : divorced/separated/married,None,4,if not real estate : building society savings ...,57,None,rent,2,skilled employee /official,1,"yes, registered under the customers name",yes
422,A12,12,critical account / other credits existing (not...,car (new),958,< 100DM,4 <= and < 7 years,2,male : single,None,3,real estate,47,None,own,2,unskilled-resident,2,None,yes
778,A14,36,critical account / other credits existing (not...,car (used),5711,>= 1000 DM,>= 7 years,4,male : single,None,2,"if not others options : car or other, not in s...",38,None,own,2,management/self-employed/highly qualified empl...,1,"yes, registered under the customers name",yes


In [61]:
# Multi_index to index

#x_train_bin = pd.DataFrame(x_train_bin.to_records())
#x_test_bin = pd.DataFrame(x_test_bin.to_records())
#x_train_bin = x_train_bin.drop("index", axis = 1)
#x_test_bin = x_test_bin.drop("index", axis = 1)

### Rule Induction

In [62]:
start_time = time.time()
print('Starting training for', CONFIG['ALGO'])

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    estimator = Ripper()
    estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    estimator = lw.RIPPER()
    estimator.fit(x_train_bin, y_train,class_feat=CONFIG["TARGET_LABEL"] , pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'R2N':
    estimator = algo.R2Nalgo(n_seeds=3, max_epochs=100, decay_rate=0.998, coef = 10**-3, normalize_num=False)
    estimator.fit(x_train_bin, y_train)
    
elif CONFIG['ALGO'] == 'CORELS':
    estimator = CorelsClassifier(n_iter=10000, 
                      # feautres per statement
                     c = 0.0001 # Higher values penalise longer rulelists
                    )
    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
    
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('Training time: ' + str(end_time - start_time))

Starting training for R2N
0 numerical features and  20 categorical features
---------
61 categorical dummies
---------


TypeError: 'NoneType' object is not subscriptable

In [ ]:
x_train_bin

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
2557,2105.0,-2.289565,-0.480260,0.818685,-1.706423,0.822102,-1.660326,0.944047,-0.541765,1.323156,...,-0.831985,-0.210837,0.914737,0.867888,0.422969,0.310584,-0.781488,0.392241,-0.147757,1.00
247823,153702.0,-0.313717,-4.064342,-3.398445,0.704011,0.101662,1.529848,1.551670,-0.036774,0.015829,...,2.142593,0.853186,-0.091941,-0.936215,-0.833081,-0.498728,0.651183,-0.290331,0.110360,1194.28
152342,97283.0,-1.809763,-0.567439,2.265186,-0.960318,-1.212537,1.516493,-1.417176,0.903421,1.961027,...,-0.554004,-0.509915,-0.424978,-0.268621,0.010121,0.466862,0.835540,-0.062385,0.088079,75.00
103385,68628.0,1.192319,0.178575,0.141491,0.459628,-0.049959,-0.112122,-0.163883,0.155740,-0.067566,...,-0.149985,-0.240464,-0.739862,0.116799,-0.373837,0.125470,0.130126,-0.016956,0.011937,1.98
8771,11951.0,-0.963451,0.700311,1.097333,-1.547626,0.669966,0.513533,0.333683,0.270900,1.381880,...,0.122458,-0.279519,-0.470181,-0.124037,-1.388839,-0.237453,0.785347,0.349708,0.216207,37.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,75618.0,1.173488,0.100792,0.490512,0.461596,-0.296377,-0.213165,-0.165254,0.119221,-0.114199,...,-0.157534,-0.186027,-0.574283,0.161405,-0.006140,0.091444,0.109235,-0.020922,0.003967,1.98
259178,159000.0,-0.775981,0.144023,-1.142399,-1.241113,1.940358,3.912076,-0.466107,1.360620,0.400697,...,-0.295730,0.037078,-0.019575,0.241830,0.682820,-1.635109,-0.770941,0.066006,0.137056,89.23
131932,79795.0,-0.146609,0.992946,1.524591,0.485774,0.349308,-0.815198,1.076640,-0.395316,-0.491303,...,0.007155,0.052649,0.354089,-0.291198,0.402849,0.237383,-0.398467,-0.121139,-0.196195,3.94
146867,87931.0,-2.948638,2.354849,-2.521201,-3.798905,1.866302,2.727695,-0.471769,2.217537,0.580199,...,0.417396,-0.332759,-1.047514,0.143326,0.678869,0.319710,0.426309,0.496912,0.335822,1.00


In [ ]:
y_train

2557      0
247823    0
152342    0
103385    0
8771      0
         ..
119879    0
259178    0
131932    0
146867    0
121958    0
Name: Class, Length: 199364, dtype: int64

In [ ]:
CONFIG['POS_CLASS']

1

### Evaluation

In [ ]:
acc_list = []
prec_list = []
%store -r acc_list
%store -r prec_list

if CONFIG['TYPE'] == 'BINARY':
    y_pred = estimator.predict(x_test_bin)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred, adjusted=True))
    print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
    print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, y_pred))

    acc_list.append(recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    prec_list.append(precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('Training time: ' + str(end_time - start_time))
    %store acc_list
    %store prec_list
   

    


Accuracy: 0.9984082955888721
Balanced accuracy: 0.0
Precision: 0.0
Recall: 0.0
F1 0.0


C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2_clone\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2_clone\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


ConfusionMatrix [[85307     0]
 [  136     0]]
F-2 0.0
Mathhews 0.0
Training time: 1164.0330731868744
Stored 'acc_list' (list)
Stored 'prec_list' (list)


In [ ]:
y_test.value_counts()

0    85307
1      136
Name: Class, dtype: int64

In [ ]:
CONFIG['POS_CLASS']

1

In [ ]:
CONFIG['BINARIZER']

'NATIVE'

In [ ]:
CONFIG['ALGO']

'R2N'

In [ ]:
df[CONFIG["TARGET_LABEL"]].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [ ]:
CONFIG['POS_CLASS']

1

In [ ]:
präds = []
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)
elif CONFIG['ALGO'] == 'BRCG':
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
elif CONFIG['ALGO'] == 'RIPPER':
    print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    print('Rule set:')
    # print(estimator.rule_list_to_pretty_string())
    # print(estimator.export_rules_to_trxf_dnf_ruleset(CONFIG['POS_CLASS']))
    rule_set_list = []
    rule_set = estimator.export_rules_to_trxf_dnf_ruleset(CONFIG['POS_CLASS'])
    conjunctions = rule_set.conjunctions
    for c in conjunctions:
        conjunction_dict = {}
        predicates = c.predicates
        for p in predicates:
            name = str(p.feature) + str(p.relation)
            value = p.value
            conjunction_dict[name] = value
        rule_set_list.append(conjunction_dict)
    # print(rule_set_list)
    rule_set_df = pd.DataFrame(rule_set_list)
    print(rule_set_df.head())
    
    #print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    #print('Rule set:')
    #print(estimator.rule_list_to_pretty_string())
    # Get predicates --> if Key Error Switch 0 to 1 or 1 to 0
    #for i in range(len(estimator.rule_map[0])):
    #    print(len(estimator.rule_map[0][i]))
    #    präds.append(len(estimator.rule_map[0][i]))
    #print("Sum Prädikate:", sum(präds))
elif CONFIG['ALGO'] == 'CORELS':
    r_length = len(estimator.rl().rules)
    print("Rule Length:", r_length)
    # Get predicates
    for i in range(len(estimator.rl().rules[0]["antecedents"])):
        an = len(estimator.rl().rules[i]["antecedents"])
        präds.append(len(estimator.rl().rules[i]["antecedents"]))
        print(f"Antecedents Length Rule {i}:" , an)
    print("Sum Prädikate:", sum(präds))
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    print("Rule Length:", len(estimator.ruleset_))

# uncomment the following line for a full optimized view of the model as data frame for GLRM rules
# explanation.style

In [ ]:
estimator.rule_map_to_intervals_list()

AttributeError: 'R2Nalgo' object has no attribute 'rule_map_to_intervals_list'

In [ ]:
estimator.rule_map[CONFIG['POS_CLASS']]

[[5_0_41958.0_-1, 2_0_525036.0_-1, 1_1_133066.0_-1],
 [1_1_154544.0_-1, 2_0_786122.0_-1, 5_0_47581.0_-1],
 [2_0_220052.0_-1, 1_1_54242.0_-1, 5_0_17756.0_-1],
 [2_0_990363.0_-1, 1_1_219984.0_-1, 5_0_66944.0_-1],
 [1_1_95850.0_-1, 2_0_518574.0_-1, 5_0_30080.0_-1],
 [5_0_49096.0_-1, 2_0_842450.0_-1, 1_1_160598.0_-1],
 [2_0_161856.0_-1, 5_0_11875.0_-1, 1_1_33849.0_-1],
 [2_0_995714.0_-1, 1_1_261448.0_-1, 5_0_79885.0_-1],
 [1_1_84388.0_-1, 2_0_959464.0_-1, 5_0_27325.0_-1, 4_1_0.03055418433499172_-1],
 [2_0_996392.0_-1, 5_0_60519.0_-1, 1_1_197486.0_-1],
 [1_1_114246.0_-1, 3_1_35.0_-1, 5_0_36334.0_-1],
 [2_0_618765.0_-1, 1_1_13653.0_-1, 5_0_5833.0_-1],
 [1_1_142294.0_-1, 3_1_215.0_-1, 5_0_44283.0_-1],
 [2_0_310409.0_-1, 1_1_65680.0_-1, 5_0_20816.0_-1],
 [1_1_169315.0_-1, 2_0_999609.0_-1, 5_0_51505.0_-1],
 [2_0_999609.0_-1, 1_1_246834.0_-1, 5_0_75734.0_-1],
 [1_1_24731.0_-1, 2_0_958562.0_-1, 5_0_8225.0_-1],
 [2_0_990012.0_-1, 1_1_208082.0_-1, 5_0_63367.0_-1],
 [2_0_986609.0_-1, 5_0_81889.0_-1,

In [ ]:
len(estimator.rule_map.values())


1

In [ ]:
estimator.rule_map[1][i]

KeyError: 1